## Imports

In [99]:
import pandas as pd
from datetime import datetime as dt
from hbt import utils_hbt as uh
from pathlib import Path

## Settings

In [100]:
dt_format = '%d.%m.%Y %H:%M:%S'
dt_start = dt.strptime("22.04.2022 00:00:00", dt_format)
dt_end = dt.strptime("01.09.2022 00:00:00", dt_format)

path_pwdata = Path(r"Q:\Projekte\10000-\10208.10 Tangente Zug"
    r"\05 Berechnungen Grundlagen\Pumpwerken\Daten_Pumpwerken")
save_dir = Path(r"Q:\Projekte\10000-\10208.10 Tangente Zug"
    r"\05 Berechnungen Grundlagen\Pumpwerken")
path_rain_ms = Path(r"Q:\Projekte\10000-\10208.10 Tangente Zug"
    r"\05 Berechnungen Grundlagen\Regendaten\Cham Niederschlag"
    r"\CHZ_Niederschlagsumme.csv")
path_rain_am = Path(r"Q:\Projekte\10000-\10208.10 Tangente Zug"
    r"\05 Berechnungen Grundlagen\Regendaten\data_Baar.csv")

pw_info = {
    'Pumpwerk Sternen': {
        'pumpen': [10, 10] # l/s
    },
    'Pumpwerk UF Zugerstrasse': {
        'pumpen': [50, 50] # l/s
    },
    'Pumpwerk Neufeld': {
        'pumpen': [60, 60, 60] # l/s
    },
    'Pumpwerk Südstrasse': {
        'pumpen': [55, 55, 55] # l/s
    },
    'Pumpwerk SABA': {
        'pumpen': [7.5, 7.5] # l/s
    }
}

## Read data

In [101]:
discharge_in = uh.hbtdb_get_data(917, dt_from=dt_start, dt_to=dt_end)

list_pwpaths = list(path_pwdata.glob('*'))
list_pwpaths = list_pwpaths[3:]
dt_format = '%d.%m.%Y %H:%M'
date_parser = lambda x: pd.to_datetime(x, format=dt_format)
df_pw = pd.DataFrame(columns=["Beschrieb","Zeitstempel","Wert"])

for pw_path in list_pwpaths:
    df_temp = pd.read_csv(pw_path, delimiter=';',
        names=["Beschrieb","Zeitstempel","Wert"], parse_dates=[1],
        date_parser=date_parser)
    df_pw = pd.concat([df_pw, df_temp])
del df_temp
df_pw.set_index('Zeitstempel', inplace=True)

mask = (df_pw.index >= dt_start) & (df_pw.index < dt_end)
df_pw = df_pw[mask]

In [102]:
discharge_out = uh.hbtdb_get_data(918, dt_from=dt_start, dt_to=dt_end)

In [103]:
ts_rain_ms = uh.read_raindata_meteoschweiz(path_rain_ms)
mask = (ts_rain_ms.index >= dt_start) & (ts_rain_ms.index < dt_end)
ts_rain_ms = ts_rain_ms[mask]
print(ts_rain_ms.sum())

ts_rain_am = uh.read_raindata_agrometeo(path_rain_am, resolution='10min')
mask = (ts_rain_am.index >= dt_start) & (ts_rain_am.index < dt_end)
ts_rain_am = ts_rain_am[mask]
print(ts_rain_am.sum())

rain [mm/10min]    422.7
dtype: float64
BAAR - Niederschlag (mm oder Liter/m2)    475.4
dtype: float64


## Analysis

In [104]:
df_betrieb = df_pw[df_pw['Beschrieb'].str.contains('Betrieb')]
index = df_betrieb['Beschrieb'].unique()
pw_tot_v = pd.DataFrame(index=index, columns=['tot vol [m3]'])
pw_tot_s = pd.DataFrame(index=index, columns=['tot time [s]'])

for pw, info in pw_info.items():
    df_temp1 = df_betrieb[df_betrieb['Beschrieb'].str.contains(pw)]
    for i, el in enumerate(df_temp1['Beschrieb'].unique()):
        df_temp2 = df_temp1[df_temp1['Beschrieb']==el]
        df_temp2.sort_index(inplace=True)
        idx_0 = df_temp2[df_temp2['Wert']==0].index
        idx_1 = df_temp2[df_temp2['Wert']==1].index
        tot_s = (idx_0 - idx_1).sum().total_seconds()
        tot_v = tot_s * info['pumpen'][i] / 1e3
        pw_tot_v.loc[el] = tot_v
        pw_tot_s.loc[el] = tot_s

pw_tot_v.sort_index(inplace=True)
pw_tot_s.sort_index(inplace=True)

save_path = save_dir / 'PW_Volumen_220422-220901.csv'
pw_tot_v.to_csv(save_path, sep=';')

save_path = save_dir / 'PW_Zeiten_220422-220901.csv'
pw_tot_s.to_csv(save_path, sep=';')

In [105]:
discharge_in, uniques = uh.ts_time_intervals(discharge_in)
tot_in1 = discharge_in['wert'].sum() * 60 / 1e3
print(tot_in1)

tot_in2 = (discharge_in['wert'] * 
    discharge_in['delta_dt'].dt.total_seconds()).sum() / 1e3
print(tot_in2)

tot_out = discharge_out['wert'].sum() * 60 / 1e3
print(tot_out)

21451.444241999994
21456.2066407
17453.40615
